In [2]:
# app.py

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ✅ Use a supported model
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_response(prompt, max_length=1024):
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=max_length,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response[len(prompt):].strip()
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

def disease_prediction(symptoms):
    prompt = (
        "You are a helpful medical assistant.\n\n"
        f"Patient reports the following symptoms: {symptoms}\n\n"
        "List possible medical conditions and general medication suggestions. "
        "Clearly state that this is not a medical diagnosis.\n\n"
        "**IMPORTANT: This is for informational purposes only. Consult a healthcare professional.**\n\n"
        "Analysis:"
    )
    return generate_response(prompt, max_length=1200)

def treatment_plan(condition, age, gender, medical_history):
    prompt = (
        "You are a helpful medical assistant.\n\n"
        f"Patient Information:\n- Condition: {condition}\n- Age: {age}\n- Gender: {gender}\n- Medical History: {medical_history}\n\n"
        "Provide a personalized treatment plan including general medication, home remedies, and lifestyle tips. "
        "Add a disclaimer about consulting a doctor.\n\n"
        "**IMPORTANT: This is for informational purposes only. Consult a healthcare professional.**\n\n"
        "Treatment Plan:"
    )
    return generate_response(prompt, max_length=1200)

with gr.Blocks() as app:
    gr.Markdown("# 🩺 Medical AI Assistant")
    gr.Markdown("**Disclaimer: This is for informational purposes only. Always consult healthcare professionals.**")

    with gr.Tabs():
        with gr.TabItem("Disease Prediction"):
            with gr.Row():
                with gr.Column():
                    symptoms_input = gr.Textbox(label="Enter Symptoms", lines=4, placeholder="e.g., fever, headache...")
                    predict_btn = gr.Button("Analyze Symptoms")
                with gr.Column():
                    prediction_output = gr.Textbox(label="Possible Conditions & Recommendations", lines=20)
            predict_btn.click(disease_prediction, inputs=symptoms_input, outputs=prediction_output)

        with gr.TabItem("Treatment Plan"):
            with gr.Row():
                with gr.Column():
                    condition_input = gr.Textbox(label="Medical Condition", lines=2)
                    age_input = gr.Number(label="Age", value=30)
                    gender_input = gr.Dropdown(choices=["Male", "Female", "Other"], label="Gender")
                    history_input = gr.Textbox(label="Medical History", lines=3)
                    plan_btn = gr.Button("Generate Treatment Plan")
                with gr.Column():
                    plan_output = gr.Textbox(label="Personalized Treatment Plan", lines=20)
            plan_btn.click(treatment_plan, inputs=[condition_input, age_input, gender_input, history_input], outputs=plan_output)

app.launch(share=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.